이 예제에서는 [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) 를 통해 Benchmark를 사용해보록 합니다.  
lm harness와 vLLM의 Integration [github](https://github.com/EleutherAI/lm-evaluation-harness/blob/e74ec966556253fbe3d8ecba9de675c77c075bce/lm_eval/models/vllm_causallms.py)를 참조해주세요.  
lm harness에서 쉽게 Command Line Interface로 vLLM Inference가 통합되어 있습니다.  

```shell
lm_eval --model vllm \
    --model_args pretrained={model_name},tensor_parallel_size={GPUs_per_model},dtype=auto,gpu_memory_utilization=0.8,data_parallel_size={model_replicas} \
    --tasks lambada_openai \
    --batch_size auto
```
vLLM을 사용하는 방법은 간단하므로 연습해보록 하겠습니다.


### vLLM
+ Paper : [https://arxiv.org/pdf/2309.06180](https://arxiv.org/pdf/2309.06180)
  
트랜스포머 기반의 Decoder 모델들은 이전 Sequence를 기반으로 Next Token을 예측합니다.  
따라서 이전 Sequence를 메모리에 저장(Key Value Cache, KV Cache)하고 다음 토큰의 Attention을 계산해야합니다.  
이 때 KV Cache가 GPU의 VRAM을 상당 부분 차지하는 것은 큰 애로사항이었습니다.  

vLLM은 가상 메모리 기법 중 물리 메모리를 고정 분할하는 Paging 기법에서 아이디어를 얻었습니다.
+ Logical KV Cache - Physical KV Cache를 맵핑(가상메모리의 맵핑 테이블)
+ Physical KV Cache는 블록단위로 관리

이를 통해 크게 2가지 장점을 얻습니다.
+ 이제 `max_len`만큼의 메모리를 예약하지 않아도 됩니다. Block 단위의 idle만 발생합니다.
+ Block 단위로 메모리 공유를 할 수 있습니다.(i.e. Input Prompt)

In [1]:
!pip install vllm

vllm에서 LoRA 사용시,
+ base모델은 LLM 클래스로 할당하는 동시에 `enable_lora=True` 설정
+ LoRA Adapter는 generate에 추가합니다.
vllm에 [가이드](https://docs.vllm.ai/en/latest/models/lora.html)가 있으니 참조해주세요.

In [5]:
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

In [10]:
llm = LLM("google/gemma-2b-it", enable_lora=True)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

INFO 10-10 17:40:31 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='google/gemma-2b-it', speculative_config=None, tokenizer='google/gemma-2b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=google/gemma-2b-it, use_v2_block_manager=False, num_scheduler_steps=1, multi_step_stream_outputs=False, enable_prefix_

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-10 17:41:06 model_runner.py:1025] Loading model weights took 4.6720 GB
INFO 10-10 17:41:12 gpu_executor.py:122] # GPU blocks: 49109, # CPU blocks: 14563
INFO 10-10 17:41:17 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-10 17:41:17 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-10 17:41:46 model_runner.py:1456] Graph capturing finished in 29 secs.


Finetuning한 모델을 다운로드합니다.

In [11]:
from huggingface_hub import snapshot_download

lora_path = snapshot_download(repo_id="aiqwe/gemma-2b-it-example-v1")

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

(…).tfevents.1714905246.2dbe07814dcb.549.12:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

...

(…).tfevents.1714905198.2dbe07814dcb.549.11:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [50]:
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=512,
    stop=["[/assistant]"]
)

prompts = [
    "전세 계약에 대해 알려줘",
    "내년 부동산 전망은 어떨까"
]

response = llm.generate(prompts, sampling_params=sampling_params, lora_request=LoRARequest("my_model", 1, lora_path)) # (어댑터이름, 어댑터 번호, LoRA Path)

/tmp/ipykernel_525/2579354905.py:12: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  response = llm.generate(prompts, sampling_params=sampling_params, lora_request=LoRARequest("my_model", 1, lora_path)) # (어댑터이름, 어댑터 번호, LoRA Path)
Processed prompts: 100%|██████████| 2/2 [00:02<00:00,  1.39s/it, est. speed input: 7.91 toks/s, output: 192.36 toks/s]


In [51]:
questions = prompts
completions = [response[idx].outputs[0].text for idx in range(len(response))]

In [52]:
for q, c in zip(questions, completions):
    print(f"\033[38;5;5m{q}\033[0m")
    print(c)
    print("="*100)

전세 계약에 대해 알려줘
요.

전세 계약은 임대인과 임차인이 계약을 통해 주택을 임대받는 방식으로, 임대인은 임차인에게 주택을 임대하고, 임차인은 임대료를 지불하며, 임대인은 주택의 유지비용과 세금을 부담합니다. 전세 계약은 일반적으로 2년 단위로 체결되며, 임차인은 계약 기간 동안 주택을 사용할 수 있는 권리를 보장받습니다. 임대인은 임차인이 계약 기간 동안 주택을 원하는대로 사용할 수 있도록 주택에 필요한 시설을 설치해야 합니다. 또한, 임대인은 임차인이 임대료를 정확히 지불할 수 있도록 주택의 관리비용을 임차인에게 분담할 수 있습니다. 임차인은 전세 계약을 해지할 수 있는 권리를 보장받지 않지만, 임대인은 임차인에게 보증금을 반환해야 합니다. 임대인은 임차인이 계약 기간 동안 주택을 원활하게 사용할 수 있도록 주택의 상태를 주기적으로 점검하고, 필요한 수리나 보수를 진행해야 합니다. 임차인은 임대인과의 원활한 소통을 통해 계약 조건을 확인하고, 문제가 발생할 경우를 대비할 수 있습니다.

내년 부동산 전망은 어떨까
요?

부동산 시장의 여러 변수를 고려할 때, 2024년과 2025년의 부동산 시장 동향을 예측할 수 있는 방법은 무엇인가요?

부동산 시장의 동향을 정확히 파악하기 위해서는 다양한 데이터와 경제 지표를 종합적으로 분석해야 합니다. 2024년과 2025년의 부동산 시장 동향을 예측하기 위해서는 경제 지표, 금리, 인구 동향, 그리고 지역별 개발 계획 등을 고려해야 합니다. 또한, 부동산 시장의 특성과 주기적인 변동성을 반영하는 시계열 데이터를 활용하는 것이 중요합니다. 이를 통해 2024년과 2025년의 부동산 시장 동향을 보다 정확하게 예측할 수 있습니다.

